<a href="https://colab.research.google.com/github/GenAIUnplugged/pytorch/blob/main/pytorch_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import torch
import torchvision
import numpy as nn
import torch.nn as nn

In [73]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [74]:
train_data = torchvision.datasets.CIFAR10(root="./data",train=True,download=True,transform=torchvision.transforms.ToTensor())

In [75]:
test_data = torchvision.datasets.CIFAR10(root="./data",train=False,download=True,transform=torchvision.transforms.ToTensor())

In [76]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=32,shuffle=True,num_workers=2,pin_memory=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=32,shuffle=False,num_workers=2,pin_memory=True)

In [77]:
len(next(iter(train_loader))[1])

32

In [78]:
next(iter(train_loader))[1]

tensor([5, 6, 0, 6, 7, 6, 3, 4, 4, 3, 6, 7, 8, 1, 6, 7, 2, 8, 6, 4, 0, 1, 8, 5,
        0, 5, 4, 2, 5, 5, 2, 3])

In [79]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.nn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(in_features=128*4*4, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10)  # No ReLU here; logits go to CrossEntropyLoss
        )

    def forward(self, x):
        return self.nn(x)


In [80]:
model=NN().to(device)

In [81]:
model.state_dict()

OrderedDict([('nn.0.weight',
              tensor([[[[ 5.0041e-02, -1.4669e-01,  2.7948e-02],
                        [-3.9499e-02, -7.1162e-02, -1.3493e-01],
                        [-1.2138e-01, -1.3053e-01,  1.0201e-01]],
              
                       [[ 1.0409e-01, -6.4307e-02, -7.5629e-02],
                        [ 1.4068e-01,  1.0576e-01,  1.5256e-02],
                        [-4.2271e-02, -1.5149e-02,  5.5081e-02]],
              
                       [[ 1.3870e-01,  6.3487e-03, -1.4257e-01],
                        [ 9.9596e-02, -8.9939e-02, -7.8294e-02],
                        [-1.7041e-01,  7.9427e-02, -1.3911e-01]]],
              
              
                      [[[ 1.8121e-01,  1.5261e-01,  1.6168e-01],
                        [-9.1037e-02, -1.7710e-01,  1.1725e-01],
                        [ 8.9237e-02,  5.8643e-02,  9.7549e-02]],
              
                       [[-1.7518e-01,  1.5169e-01, -1.5804e-01],
                        [-1.4414e-01,  1.2153e

In [82]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)

In [83]:
def train(model,device,criterion,optimizer,train_loader):
  epochs = 10

  model.train()
  for epoch in range(epochs):
    loss=0
    total_batches = 0
    total_accuracy = 0
    for idx,(images,labels) in enumerate(train_loader):
      images,labels = images.to(device),labels.to(device)
      train_logits = model(images)
      current_loss = criterion(train_logits,labels)
      y_pred=torch.softmax(train_logits,dim=1).argmax(dim=1)
      total_accuracy+=(y_pred==labels).sum().item()
      loss += current_loss.item()
      total_batches+=1
      optimizer.zero_grad()
      current_loss.backward()
      optimizer.step()
    total_loss = loss / total_batches
    print(f"epoch:{epoch},loss={total_loss},accuracy:{total_accuracy/len(train_loader.dataset)}")

In [84]:
def test(model,device,criterion,optimizer,test_loader):
  model.eval()
  test_loss = correct = 0
  with torch.no_grad():
    for idx,(images,labels) in enumerate(test_loader):
      images,labels = images.to(device),labels.to(device)
      test_logits = model(images)
      test_loss+=criterion(test_logits, labels).item() # Sum up batch loss
      _, predicted = test_logits.max(1)
      correct += (predicted == labels).sum().item()
    test_loss /= len(test_loader.dataset) # Average loss over the dataset
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

In [85]:
device

'cuda'

In [86]:
train(model,device,criterion,optimizer,train_loader)
test(model,device,criterion,optimizer,test_loader)

epoch:0,loss=1.5257720581934533,accuracy:0.44204
epoch:1,loss=1.1062962505883005,accuracy:0.60494
epoch:2,loss=0.9214615742327384,accuracy:0.67614
epoch:3,loss=0.8025017098135774,accuracy:0.71856
epoch:4,loss=0.7163002761563504,accuracy:0.74952
epoch:5,loss=0.6480262769542287,accuracy:0.77522
epoch:6,loss=0.5871897627833709,accuracy:0.79598
epoch:7,loss=0.5378684107779084,accuracy:0.81066
epoch:8,loss=0.48556438235235916,accuracy:0.8302
epoch:9,loss=0.4459979433139699,accuracy:0.84134

Test set: Average loss: 0.0256, Accuracy: 7462/10000 (75%)

